In [1]:
import pandas as pd
import numpy as np

In [2]:
node_info = pd.read_csv("/project/biocomplexity/nssac/EpiHiperSynPop/v1.9.0/usa_va_2017_SynPop/va_persontrait_epihiper.txt", skiprows=1)

print(node_info.admin3.unique())
node_info["census_tract"] = node_info.admin3.apply(lambda row: str(row)[:-2] + "." + str(row)[-2:])
node_info["census_tract"]=node_info.census_tract.apply(lambda row: row.replace(".00", ""))
print(node_info.census_tract.unique())
node_to_census = node_info.set_index("pid")["census_tract"].to_dict() #[node_info.admin2 == 1].admin3.unique() #.info() #.admin3.value_counts()
tract_pops = node_info.census_tract.apply(lambda row: row.replace(".00", "")).value_counts() #num_replicates
print(tract_pops)

#ind=np.arange(node_info.census_tract.unique().size)


[ 90100  90200  90300 ... 370100 370200 370300]
['901' '902' '903' ... '3701' '3702' '3703']
9301       58835
102        49494
101        47491
103        45861
203        45399
           ...  
1034.01      134
450           75
109.03        70
9801          22
9802           9
Name: census_tract, Length: 1411, dtype: int64


In [3]:
tractIndex = {tract.replace(".00", ""):0 for tract in node_info.census_tract.unique() }
ind=0
for i in tractIndex:
    tractIndex[i]=ind
    ind+=1
l=node_info.census_tract.unique()
tactMat=np.zeros((node_info.census_tract.unique().size,node_info.census_tract.unique().size))
edgeFullList=pd.read_csv("/project/biocomplexity/nssac/EpiHiperSynPop/v1.9.0/usa_va_2017_SynPop/va_contact_network_config_m_5_M_40_a_1000_m-contact_0_no_lid.txt", skiprows=1)

In [4]:

for row in edgeFullList.itertuples(index=False):
    tactMat[tractIndex[node_info["census_tract"][row[2]]],tractIndex[node_info["census_tract"][row[0]]]]+=1
    
np.savetxt("file2.txt", tactMat)    

In [5]:
print(tactMat)

[[8.81780e+04 4.52540e+04 8.23300e+03 ... 7.00000e+00 6.00000e+00
  9.00000e+00]
 [4.52540e+04 1.18594e+05 1.83230e+04 ... 6.00000e+00 0.00000e+00
  4.00000e+00]
 [8.23300e+03 1.83230e+04 2.16580e+04 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 ...
 [7.00000e+00 6.00000e+00 0.00000e+00 ... 2.32620e+04 6.58900e+03
  7.78600e+03]
 [6.00000e+00 0.00000e+00 0.00000e+00 ... 6.58900e+03 2.35380e+04
  7.43900e+03]
 [9.00000e+00 4.00000e+00 1.00000e+00 ... 7.78600e+03 7.43900e+03
  2.55780e+04]]


In [88]:
tractIndexR = {v: k for k, v in tractIndex.items()}

elist=np.transpose(np.nonzero(tactMat))

elistCensusTrack=np.zeros(elist.shape)
for i in np.arange(elist.shape[0]):
    elistCensusTrack[i, 0]=tractIndexR[elist[i,0]]
    elistCensusTrack[i, 1]=tractIndexR[elist[i,1]]
    
    
print(np.transpose(tactMat[elist[:,0], elist[:,1]]))
elistCensusTrack=np.append(elistCensusTrack, tactMat[elist[:,0], elist[:,1]].reshape(tactMat[elist[:,0], elist[:,1]].shape[0], 1), axis = 1)
#tactMat[elist[:,0], elist[:,1]]


[88178. 45254.  8233. ...  7786.  7439. 25578.]


In [94]:
elistCensusTrackDataframe=pd.DataFrame(elistCensusTrack, columns = ['source','target','noofLink'])
elistCensusTrackDataframe.to_csv("/project/biocomplexity/biosurveillance/EpiHiper/preProcessing/regression/networkCensustrack.csv",index=False) 

In [92]:
elistCensusTrackDataframe

,source,target,noofLink
0,901.0,901.00,88178.0
1,901.0,902.00,45254.0
2,901.0,903.00,8233.0
3,901.0,904.00,18109.0
4,901.0,905.00,8301.0
...,...,...,...
1367967,3703.0,462.16,16.0
1367968,3703.0,464.00,14.0
1367969,3703.0,3701.00,7786.0
1367970,3703.0,3702.00,7439.0


In [80]:
elist[1000,1]

1242